### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
#
# see https://python.langchain.com/docs/modules/model_io/models/llms/custom_llm
#
from typing import Any, List, Mapping, Optional
from time import time

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

import oci
import requests
from oci.signer import Signer

In [9]:
class DS_AQUA_LLM(LLM):

    model: str = "odsc-llm" # this is a constant
    debug: bool = False

    max_tokens: int = 300
    temperature: int = 0
    frequency_penalty: int = 1
    top_p: float = 0.75
    top_k: int = 0
    config: Optional[Any] = None
    auth: Optional[Any] = None
    service_endpoint: Optional[str] = None
    compartment_id: Optional[str] = None
    timeout: Optional[int] = 10
    
    """OCI AQUA LLM model.

    To use, you should have the ``oci`` python package installed, and pass 
    named parameters to the constructor.

    Example:
        .. code-block:: python

            compartment_id = "ocid1.compartment.oc1..."
            CONFIG_PROFILE = "my_custom_profile" # or DEFAULT
            config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)
            endpoint = "https://modeldeployment.ap-mumbai-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.ap-mumbai-1.amaaaaaap77apcqa4wixlxxjcbzf4ua5kik7mwakgp3nw6fzutajcv2rsdoq/predict"
            llm = DS_AQUA_LLM(
                temperature=0, 
                config=config, 
                # compartment_id=compartment_id, 
                service_endpoint=endpoint
                )


    """

    def __init__(self, **kwargs):
        # print(kwargs)
        super().__init__(**kwargs)
        
        config=self.config
        
        self.auth = Signer(
          tenancy=config['tenancy'],
          user=config['user'],
          fingerprint=config['fingerprint'],
          private_key_file_location=config['key_file']#,
          #pass_phrase=config['pass_phrase']
            )
        
#         auth = Signer(
#           tenancy=config['tenancy'],
#           user=config['user'],
#           fingerprint=config['fingerprint'],
#           private_key_file_location=config['key_file']#,
#           #pass_phrase=config['pass_phrase']
#             )
        
        service_endpoint=self.service_endpoint

    @property
    def _llm_type(self) -> str:
        return "OCI AQUA LLM"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        # calling OCI AQUA
        tStart = time()
        
        body = {
            "model": self.model, # this is a constant
            # "prompt": "what are activation functions?",
            # "prompt": "what is 2+2?",
            "prompt": [prompt],
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
            "top_p": self.top_p,
            "top_k": self.top_k
        }

        if self.debug:
            print()
            print("The input prompt is:")
            print(prompt)
            print()
            
        print("Calling OCI AQUA...")
        headers={'Content-Type':'application/json','enable-streaming':'true', 'Accept': 'text/event-stream'}
        response = requests.post(self.service_endpoint, json=body, auth=self.auth, stream=True, headers=headers)

        tEla = time() - tStart

        if self.debug:
            print(f"Elapsed time: {round(tEla, 1)} sec...")
            print()

        # print(f"Full response: {response.json()}")
        return response.json()['choices'][0]['text'].strip()

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "config": self.config,
            "auth": self.auth,
            "service_endpoint": self.service_endpoint,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
            "frequency_penalty": self.frequency_penalty,
            "top_p": self.top_p,
            "top_k": self.top_k,
        }
    
    def __repr__(self):
        return f'Model Parameters:\n{self._identifying_params}'


In [10]:
# The OCI SDK must be installed for this example to function properly.
# Installation instructions can be found here: https://docs.oracle.com/en-us/iaas/Content/API/SDKDocs/pythonsdk.htm
import oci
import requests
from oci.signer import Signer

In [11]:
config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
print(config)

{'log_requests': False, 'additional_user_agent': '', 'pass_phrase': None, 'user': 'ocid1.user.oc1..aaaaaaaayzxvcdf4jax4gdwlnxm5p5s6ryyfjrorvshg22nezpymxyxjwnqa', 'fingerprint': '56:c0:8a:d1:3f:02:99:b7:79:63:d6:62:d3:18:46:c3', 'key_file': '/home/datascience/.oci/oci_api_key_rahul.pem', 'tenancy': 'ocid1.tenancy.oc1..aaaaaaaa5nras26nkpnlcd4g3ox6gfmzjaahxwb245stlrh7krkfrpu2nu5q', 'region': 'ap-mumbai-1'}


In [12]:
endpoint = 'https://modeldeployment.ap-mumbai-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.ap-mumbai-1.amaaaaaap77apcqa4wixlxxjcbzf4ua5kik7mwakgp3nw6fzutajcv2rsdoq/predict'

In [13]:
llm = DS_AQUA_LLM(
                temperature=0.7, 
                config=config, 
                # compartment_id=compartment_id, 
                service_endpoint=endpoint
                )

In [14]:
llm

Model Parameters:
{'config': {'log_requests': False, 'additional_user_agent': '', 'pass_phrase': None, 'user': 'ocid1.user.oc1..aaaaaaaayzxvcdf4jax4gdwlnxm5p5s6ryyfjrorvshg22nezpymxyxjwnqa', 'fingerprint': '56:c0:8a:d1:3f:02:99:b7:79:63:d6:62:d3:18:46:c3', 'key_file': '/home/datascience/.oci/oci_api_key_rahul.pem', 'tenancy': 'ocid1.tenancy.oc1..aaaaaaaa5nras26nkpnlcd4g3ox6gfmzjaahxwb245stlrh7krkfrpu2nu5q', 'region': 'ap-mumbai-1'}, 'auth': <oci.signer.Signer object at 0x7f22370bdd60>, 'service_endpoint': 'https://modeldeployment.ap-mumbai-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.ap-mumbai-1.amaaaaaap77apcqa4wixlxxjcbzf4ua5kik7mwakgp3nw6fzutajcv2rsdoq/predict', 'max_tokens': 300, 'temperature': 0, 'frequency_penalty': 1, 'top_p': 0.75, 'top_k': 0}

In [15]:
llm.invoke("what is oci?")

Calling OCI AQUA...


"Oracle Cloud Infrastructure (OCI) is a cloud computing platform and infrastructure service offering from Oracle Corporation. It provides services such as compute, storage, networking, and databases, as well as integrated applications and services for various industries and use cases. OCI is designed to run both traditional enterprise workloads and new cloud-native applications. It is built on the same technology stack as Oracle's on-premises solutions, allowing for seamless migration and integration between on-premises and cloud environments. OCI is available as a public cloud, as well as a dedicated and virtual private cloud, to meet different security and compliance requirements."

In [9]:
help(llm)

Help on DS_AQUA_LLM in module __main__ object:

class DS_AQUA_LLM(langchain_core.language_models.llms.LLM)
 |  DS_AQUA_LLM(*, name: Union[str, NoneType] = None, cache: Union[bool, NoneType] = None, verbose: bool = None, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Union[List[str], NoneType] = None, metadata: Union[Dict[str, Any], NoneType] = None, callback_manager: Union[langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, model: str = 'odsc-llm', debug: bool = False, max_tokens: int = 300, temperature: int = 0, frequency_penalty: int = 1, top_p: float = 0.75, top_k: int = 0, config: Union[Any, NoneType] = None, service_endpoint: Union[str, NoneType] = None, compartment_id: Union[str, NoneType] = None, timeout: Union[int, NoneType] = 10) -> None
 |  
 |  Base LLM abstract class.
 |  
 |  The purpose of this class is to expose a simpler interface for working
 |  with L